# **CLASIFICACIÓN DE TEXTO MULTICLASE**
Predecir el tópico al que pertence un correo electrónico

# **CONFIGURACIÓN DEL ENTORNO**

##**1. Conexión con Pyspark y Google drive:** 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9ac7c3dea6ddd377ddaea0a53e4f6db1551ccab357a1293579cfcaa2dd9f06bf
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


Pyspark
pipenv install pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Clasificacionpyspark") \
    .config("spark.ui.reverseProxy", "true") \
    .config("spark.ui.reverseProxyUrl", "http://localhost:8888") \
    .getOrCreate()

In [4]:
spark

##**2. Lectura de datos:**  Leer la data en Zona Oro que contiene el dataframe ya etiquetado.

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [6]:
# Lee el archivo CSV con el esquema especificado
df = spark.read.csv('/content/drive/MyDrive/EAFIT/Semestre 3/Minería para Grandes volúmenes de datos/Proyecto/proyecto de minería/Jobs/Repositorio de analitica/Data/3. Oro/df_etiquetado.csv', header=True)
df.show()

+--------------------+--------------+
|    corpus_Contenido|Dominant_Topic|
+--------------------+--------------+
|tardesl escribir ...|             4|
|pago factura ver ...|             2|
|ingenicorp 6 solo...|             1|
|escribir presente...|             4|
|consulta resuelto...|             4|
|empresa pagado cu...|             0|
|ver cambiado 03 f...|             0|
|creer cuenta fact...|             1|
|tal 500 dolar met...|             4|
|respuesta esperer...|             0|
|creer cuenta fact...|             1|
|creer cuenta fact...|             1|
|poder ver video v...|             1|
|estimado informar...|             0|
|tendre contar bue...|             4|
|urge contar diner...|             4|
|recordar haber ha...|             4|
|urge contar diner...|             4|
|eliminar puja fc5...|             3|
|recordar haber ha...|             4|
+--------------------+--------------+
only showing top 20 rows



In [7]:
# Contar las clases
df.groupBy('Dominant_Topic').count().show()

+--------------+-----+
|Dominant_Topic|count|
+--------------+-----+
|             3|  167|
|             0| 1821|
|             5|  296|
|             1|  560|
|             4| 2509|
|             2|  467|
+--------------+-----+



In [8]:
# Contar valores con pandas
df.toPandas()['Dominant_Topic'].value_counts()

4    2509
0    1821
1     560
2     467
5     296
3     167
Name: Dominant_Topic, dtype: int64

In [9]:
# Valores nulos
df.toPandas()['corpus_Contenido'].isnull().sum()

9

In [10]:
# Quitar valores nulos
df = df.dropna(subset=('corpus_Contenido'))

In [11]:
# Revisar valores nulos nuevamente
df.toPandas()['corpus_Contenido'].isnull().sum()

0

# **EXTRACCIÓN DE CARACTERÍSTICAS** Adaptar datos a los requerimientos del modelo

In [12]:
# metodos para extracción o transformación de características como Hashing TF, countvectorizer, IDF ...
import pyspark.ml.feature
# Lista
dir(pyspark.ml.feature)
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.feature import StringIndexer

In [13]:
# Estados del Pipeline
# 1. Tokenizar
tokenizer = Tokenizer(inputCol='corpus_Contenido',outputCol='mytokens')
# 2. Remover stopwords
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
# 3. Vectorizar con countvectorizer
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures', vocabSize=5000)
# 4. Caracteristicas vectorizadas finales
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [14]:
# Asignación de una etiqueta al tópico y agregarla al df: no sería necesario pero facilita la manipulación
labelEncoder = StringIndexer(inputCol='Dominant_Topic',outputCol='label').fit(df)
labelEncoder.transform(df).show(5)

+--------------------+--------------+-----+
|    corpus_Contenido|Dominant_Topic|label|
+--------------------+--------------+-----+
|tardesl escribir ...|             4|  0.0|
|pago factura ver ...|             2|  3.0|
|ingenicorp 6 solo...|             1|  2.0|
|escribir presente...|             4|  0.0|
|consulta resuelto...|             4|  0.0|
+--------------------+--------------+-----+
only showing top 5 rows



In [15]:
labelEncoder.labels

['4', '0', '1', '2', '5', '3']

In [16]:
# Dict of Labels
label_dict = {'Topico 4':0.0,
 'Topico 0':1.0,
 'Topico 1':2.0,
'Topico 2':3.0,
'Topico 5':4.0,
'Topico 3':5.0}

In [17]:
df.show()

+--------------------+--------------+
|    corpus_Contenido|Dominant_Topic|
+--------------------+--------------+
|tardesl escribir ...|             4|
|pago factura ver ...|             2|
|ingenicorp 6 solo...|             1|
|escribir presente...|             4|
|consulta resuelto...|             4|
|empresa pagado cu...|             0|
|ver cambiado 03 f...|             0|
|creer cuenta fact...|             1|
|tal 500 dolar met...|             4|
|respuesta esperer...|             0|
|creer cuenta fact...|             1|
|creer cuenta fact...|             1|
|poder ver video v...|             1|
|estimado informar...|             0|
|tendre contar bue...|             4|
|urge contar diner...|             4|
|recordar haber ha...|             4|
|urge contar diner...|             4|
|eliminar puja fc5...|             3|
|recordar haber ha...|             4|
+--------------------+--------------+
only showing top 20 rows



In [18]:
df = labelEncoder.transform(df)

In [19]:
df.show()

+--------------------+--------------+-----+
|    corpus_Contenido|Dominant_Topic|label|
+--------------------+--------------+-----+
|tardesl escribir ...|             4|  0.0|
|pago factura ver ...|             2|  3.0|
|ingenicorp 6 solo...|             1|  2.0|
|escribir presente...|             4|  0.0|
|consulta resuelto...|             4|  0.0|
|empresa pagado cu...|             0|  1.0|
|ver cambiado 03 f...|             0|  1.0|
|creer cuenta fact...|             1|  2.0|
|tal 500 dolar met...|             4|  0.0|
|respuesta esperer...|             0|  1.0|
|creer cuenta fact...|             1|  2.0|
|creer cuenta fact...|             1|  2.0|
|poder ver video v...|             1|  2.0|
|estimado informar...|             0|  1.0|
|tendre contar bue...|             4|  0.0|
|urge contar diner...|             4|  0.0|
|recordar haber ha...|             4|  0.0|
|urge contar diner...|             4|  0.0|
|eliminar puja fc5...|             3|  5.0|
|recordar haber ha...|          

#**ENTRENAMIENTO CON 100% DE LOS DATOS**


In [27]:
# Proporciones para la partición de datos (train, test, validation)
trainDF = df


In [28]:
trainDF.show()

+--------------------+--------------+-----+
|    corpus_Contenido|Dominant_Topic|label|
+--------------------+--------------+-----+
|tardesl escribir ...|             4|  0.0|
|pago factura ver ...|             2|  3.0|
|ingenicorp 6 solo...|             1|  2.0|
|escribir presente...|             4|  0.0|
|consulta resuelto...|             4|  0.0|
|empresa pagado cu...|             0|  1.0|
|ver cambiado 03 f...|             0|  1.0|
|creer cuenta fact...|             1|  2.0|
|tal 500 dolar met...|             4|  0.0|
|respuesta esperer...|             0|  1.0|
|creer cuenta fact...|             1|  2.0|
|creer cuenta fact...|             1|  2.0|
|poder ver video v...|             1|  2.0|
|estimado informar...|             0|  1.0|
|tendre contar bue...|             4|  0.0|
|urge contar diner...|             4|  0.0|
|recordar haber ha...|             4|  0.0|
|urge contar diner...|             4|  0.0|
|eliminar puja fc5...|             3|  5.0|
|recordar haber ha...|          

In [44]:
# Contar las clases
df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2509|
|  1.0| 1812|
|  4.0|  296|
|  3.0|  467|
|  2.0|  560|
|  5.0|  167|
+-----+-----+



# **Building the Pipeline**

In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [30]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [31]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [32]:
pipeline

Pipeline_2d488f2aa0cd

In [33]:
pipeline.stages

Param(parent='Pipeline_2d488f2aa0cd', name='stages', doc='a list of pipeline stages')

# **Entrenar con Regresión logística**

In [34]:
lr_model = pipeline.fit(trainDF)

# **Predecir nuevos correos**

In [35]:
from pyspark.sql.types import StringType

In [36]:
ex1 = spark.createDataFrame([
    ("No he recibido reportes sobre mi operación, hace 8 dias esta en estado de cobranza",StringType())
],
# Column Name
["corpus_Contenido"]

)

In [37]:
ex1.show(truncate=False)

+----------------------------------------------------------------------------------+---+
|corpus_Contenido                                                                  |_2 |
+----------------------------------------------------------------------------------+---+
|No he recibido reportes sobre mi operación, hace 8 dias esta en estado de cobranza|{} |
+----------------------------------------------------------------------------------+---+



In [38]:
# Predict
pred_ex1 = lr_model.transform(ex1)

In [39]:
pred_ex1.show()

+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    corpus_Contenido| _2|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|No he recibido re...| {}|[no, he, recibido...|[recibido, report...|(26586,[3,212,134...|(26586,[3,212,134...|[104.928355593049...|[1.0,3.4249634834...|       0.0|
+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+



In [40]:
pred_ex1.columns

['corpus_Contenido',
 '_2',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [41]:
pred_ex1.select('corpus_Contenido','rawPrediction','probability','prediction').show()

+--------------------+--------------------+--------------------+----------+
|    corpus_Contenido|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|No he recibido re...|[104.928355593049...|[1.0,3.4249634834...|       0.0|
+--------------------+--------------------+--------------------+----------+

